In [1]:
import os
import cv2
import numpy as np

In [2]:
path = os.path.abspath('../data')
path

'/home/roger/code/xkeeja/corpy-assessment/data'

In [3]:
def load_data(path):    
    g_files = []
    ng_files = []
    X, y = [], []
    
    g_dir = os.path.join(path,'train','good')
    for filename in os.listdir(g_dir):
        f = os.path.join(g_dir, filename)
        g_files.append(f)
    
    for file in g_files:
        X.append(cv2.imread(file))
        y.append(0)
        
    ng_dir = os.path.join(path,'train','not-good')
    for filename in os.listdir(ng_dir):
        f = os.path.join(ng_dir, filename)
        ng_files.append(f)
    
    for file in ng_files:
        X.append(cv2.imread(file))
        y.append(1)

    c = list(zip(X, y))
    np.random.shuffle(c)
    X, y = zip(*c)
        
    return np.array(X), np.array(y)

In [4]:
X, y = load_data(path)
X.shape, y.shape

((300, 1024, 1024, 3), (300,))

In [8]:
cv2.imshow('sample',X[99])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
np.unique(X)

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186,
       187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 19